In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings; warnings.simplefilter('ignore', FutureWarning)
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import pymongo
from pymongo import MongoClient
from avian_vocalizations.evaluator import ParamSpace, EvaluatorFactory
from hyperopt.mongoexp import MongoTrials, MongoCtrl, MongoJobs, MongoWorker
from hyperopt import hp, fmin, tpe
from vastai.api import VastClient
from toolbench.tools import print_module_versions
print_module_versions(globals())

Using TensorFlow backend.


             IPython  {'7.8.0'}
          tensorflow  {'1.14.0'}
             pymongo  {'3.9.0'}
 avian_vocalizations  {'0.0.1'}
            hyperopt  {'0.2.1'}
           toolbench  {'0.0.1'}


In [3]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


## Test tunnel to mongodb

In [4]:
instance = vast.get_instance(400808)
remote = instance.pb_remote

In [5]:
print(instance.ssh_connection_command)

ssh root@ssh4.vast.ai -p 10808 -i /home/sam/.ssh/vastai


In [6]:
print(remote['tail']('/var/log/mongodb/mongodb.log','-n2'))

2019-10-26T21:35:41.588+0000 I NETWORK  [listener] connection accepted from 127.0.0.1:55566 #5 (4 connections now open)
2019-10-26T21:35:41.588+0000 I NETWORK  [conn5] received client metadata from 127.0.0.1:55566 conn: { driver: { name: "PyMongo", version: "3.9.0" }, os: { type: "Linux", name: "Linux", architecture: "x86_64", version: "4.15.0-65-generic" }, platform: "CPython 3.6.8.final.0" }



In [10]:
with instance.get_tunnel(27017,27017) as tunn:
    mongo = MongoClient('localhost', 27017)
    print(mongo.list_database_names())

['admin', 'avian_vocalizations', 'config', 'hyperopt', 'local']


## Hot update evaluator code

In [77]:
remote.sftp.put('avian_vocalizations/evaluator.py',
                '/root/avian_vocalizations/avian_vocalizations/evaluator.py')

<SFTPAttributes: [ size=4895 uid=0 gid=0 mode=0o100644 atime=1572122598 mtime=1572123513 ]>

In [68]:
remote.sftp.put('avian_vocalizations/data.py',
                '/root/avian_vocalizations/avian_vocalizations/data.py')

<SFTPAttributes: [ size=13787 uid=0 gid=0 mode=0o100644 atime=1572122143 mtime=1572122578 ]>

In [ ]:
remote.sftp.put('avian_vocalizations/model.py',
                '/root/avian_vocalizations/avian_vocalizations/model.py')

## Update code repo

In [10]:
with remote.cwd('/root/avian_vocalizations'):
    print(remote['git']('pull'))

Updating 91eb747..2308022
Fast-forward
 Hyperopt.ipynb                   |  73 +++---
 Run Hyperopt Job.ipynb           | 372 +++++++++++++++++++++++++++
 Setup Hyperopt Instance.ipynb    | 370 +++++++++++++++++++++++++++
 SetupMongoInstance.ipynb         | 537 +++++++++++++--------------------------
 avian_vocalizations/data.py      |  10 +-
 avian_vocalizations/evaluator.py |  60 +++--
 6 files changed, 986 insertions(+), 436 deletions(-)
 create mode 100644 Run Hyperopt Job.ipynb
 create mode 100644 Setup Hyperopt Instance.ipynb



## Run some trials

In [7]:
tunnel = instance.get_tunnel(27017, 27017)

In [8]:
trials = MongoTrials('mongo://localhost:27017/avian_vocalizations/jobs', exp_key='test_experiment')

In [10]:
hp_space = ParamSpace(n_frames=128, dropout_rate=hp.uniform('dropout',0,.5),batch_size=64)

evaluator = EvaluatorFactory(n_splits=3, n_epochs=10, data_dir='data')
best = fmin(evaluator, hp_space, trials=trials, algo=tpe.suggest, max_evals=1 )

  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

no last_id found, re-trying


100%|██████████| 1/1 [00:03<00:00,  3.05s/it, best loss: ?]


Exception: There are no evaluation tasks, cannot return argmin of task losses.

In [9]:
# trials.delete_all()

In [11]:
import numpy as np
np.log()

<function numpy.where>

## Cleanup

In [ ]:
tunnel.close()